# Pointcloud-to-Graph Geodesic Lifting Tutorial

***
This notebook shows how to import a dataset, with the desired lifting, and how to run a neural network using the loaded data.

The notebook is divided into sections:

- [Loading the dataset](#loading-the-dataset) loads the config files for the data and the desired tranformation, createsa a dataset object and visualizes it.
- [Loading and applying the lifting](#loading-and-applying-the-lifting) defines a simple neural network to test that the lifting creates the expected incidence matrices.
- [Create and run a simplicial nn model](#create-and-run-a-simplicial-nn-model) simply runs a forward pass of the model to check that everything is working as expected.

***
***

Note that for simplicity the notebook is setup to use a simple graph. However, there is a set of available datasets that you can play with.

To switch to one of the available datasets, simply change the *dataset_name* variable in [Dataset config](#dataset-config) to one of the following names:

* cocitation_cora
* cocitation_citeseer
* cocitation_pubmed
* MUTAG
* NCI1
* NCI109
* PROTEINS_TU
* AQSOL
* ZINC
***

### Imports and utilities

In [1]:
# With this cell any imported module is reloaded before each cell execution
%load_ext autoreload
%autoreload 2
from modules.data.load.loaders import PointCloudLoader
from modules.data.preprocess.preprocessor import PreProcessor
from modules.utils.utils import (
    describe_data,
    load_dataset_config,
    load_model_config,
    load_transform_config,
)

## Loading the Dataset

Here we just need to spicify the name of the available dataset that we want to load. First, the dataset config is read from the corresponding yaml file (located at `/configs/datasets/` directory), and then the data is loaded via the implemented `Loaders`.


In [2]:
dataset_name = "gudhi_bunny"
dataset_config = load_dataset_config(dataset_name)
loader = PointCloudLoader(dataset_config)


Dataset configuration for gudhi_bunny:

{'data_domain': 'pointcloud',
 'data_type': 'gudhi',
 'data_name': 'gudhi_bunny',
 'data_dir': 'datasets/pointcloud/gudhi',
 'task': 'regression',
 'loss_type': 'mse',
 'monitor_metric': 'mae'}


We can then access to the data through the `load()`method:

In [3]:
dataset = loader.load()
describe_data(dataset)


Dataset only contains 1 sample:
 - Graph with 35947 vertices and 0 edges.
 - Features dimensions: [0, 0]



## Loading and Applying the Lifting

In this section we will instantiate the lifting we want to apply to the data. For this example the clique lifting was chosen. For a clique of n nodes the algorithm for $m=3,...,max(n, complex\_dim)$ will create simplicials for every possible combinations containing m nodes of the clique. $complex\_dim$ is a parameter of the lifting. This is a deterministic lifting, based on connectivity, that does not modify the initial connectivity of the graph. The problem of extracting all the cliques in a graph is NP-hard, on in some formulaitons NP-complete (clique decision problem). The computational complexity of this algorithm is $O(n^k k^2)$[[1]](https://www.sciencedirect.com/science/article/pii/S0019995885800413), where $n$ is the number of nodes in the graph and $k$ is the highest clique dimension considered.

***
[[1]](https://www.sciencedirect.com/science/article/pii/S0019995885800413) Cook, S. A. (1985). A taxonomy of problems with fast parallel algorithms. Information and control, 64(1-3), 2-22.
***

For simplicial complexes creating a lifting involves creating a `SimplicialComplex` object from topomodelx and adding simplices to it using the method `add_simplices_from`. The `SimplicialComplex` class then takes care of creating all the needed matrices.

Similarly to before, we can specify the transformation we want to apply through its type and id --the correxponding config files located at `/configs/transforms`. 

Note that the *tranform_config* dictionary generated below can contain a sequence of tranforms if it is needed.

This can also be used to explore liftings from one topological domain to another, for example using two liftings it is possible to achieve a sequence such as: graph -> simplicial complex -> hypergraph. 

In [4]:
# Define transformation type and id
transform_type = "liftings"
# If the transform is a topological lifting, it should include both the type of the lifting and the identifier
transform_id = "pointcloud2graph/geodesic_lifting"

# Read yaml file
transform_config = {
    "lifting": load_transform_config(transform_type, transform_id)
    # other transforms (e.g. data manipulations, feature liftings) can be added here
}


Transform configuration for pointcloud2graph/geodesic_lifting:

{'transform_type': 'lifting',
 'transform_name': 'GeodesicLifting',
 'feature_lifting': 'ProjectionSum',
 'distance': 1.0}


We than apply the transform via our `PreProcesor`:

In [5]:
lifted_dataset = PreProcessor(dataset, transform_config, loader.data_dir)
describe_data(lifted_dataset)

Processing...


  0%|          | 0/35947 [00:00<?, ?it/s]

ValueError: invalid literal for int() with base 10: 'edges'

## Create and Run a Simplicial NN Model

In this section a simple model is created to test that the used lifting works as intended. In this case the model uses the `up_laplacian_1` and the `down_laplacian_1` so the lifting should make sure to add them to the data.

In [ ]:
from modules.models.simplicial.san import SANModel

model_type = "simplicial"
model_id = "san"
model_config = load_model_config(model_type, model_id)

model = SANModel(model_config, dataset_config)


Model configuration for simplicial SAN:

{'in_channels': None,
 'hidden_channels': 32,
 'out_channels': None,
 'n_layers': 2,
 'n_filters': 2,
 'order_harmonic': 5,
 'epsilon_harmonic': 0.1}


In [ ]:
y_hat = model(lifted_dataset.get(0))

If everything is correct the cell above should execute without errors. 